# Логистическая регрессия. Практика

В этом задании вам предлагается спрогнозировать, купит клиент велосипед или нет, обучив логистическую регрессию.

In [404]:
# подключим библиотеки
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

import warnings
warnings.filterwarnings("ignore")

In [405]:
# считаем данные
data = pd.read_csv('https://raw.githubusercontent.com/evgpat/edu_stepik_practical_ml/main/datasets/bike_buyers_clean.csv')

In [406]:
# выводим первые 5 строк датафрейма
data.head(5)

,ID,Marital Status,Gender,Income,Children,Education,Occupation,Home Owner,Cars,Commute Distance,Region,Age,Purchased Bike
0,12496,Married,Female,40000,1,Bachelors,Skilled Manual,Yes,0,0-1 Miles,Europe,42,No
1,24107,Married,Male,30000,3,Partial College,Clerical,Yes,1,0-1 Miles,Europe,43,No
2,14177,Married,Male,80000,5,Partial College,Professional,No,2,2-5 Miles,Europe,60,No
3,24381,Single,Male,70000,0,Bachelors,Professional,Yes,1,5-10 Miles,Pacific,41,Yes
4,25597,Single,Male,30000,0,Bachelors,Clerical,No,0,0-1 Miles,Europe,36,Yes


In [407]:
# смотрим размер датафрейма
data.shape

(1000, 13)

Выведите статистики по категориальным признакам, чтобы посмотреть, сколько категорий в каждом категориальном (нечисловом) признаке.

Для этого можно воспользоваться методом `describe` из библиотеки pandas со значением параметра `include = 'object'`.

**Вопрос:** в каком категориальном признаке встречаются три различных значения?

In [408]:
data.describe(include='object')

,Marital Status,Gender,Education,Occupation,Home Owner,Commute Distance,Region,Purchased Bike
count,1000,1000,1000,1000,1000,1000,1000,1000
unique,2,2,5,5,2,5,3,2
top,Married,Male,Bachelors,Professional,Yes,0-1 Miles,North America,No
freq,539,509,306,276,685,366,508,519


Закодируйте все категориальные столбцы с двумя категориями следующим образом:  
самая часто встречающаяся категория превращается в 1, другая в 0.

In [409]:
for column in data.columns:
    if data[column].dtype == 'object' and data[column].nunique() == 2:
        unique_vals = data[column].unique()
        # print(unique_vals)
        most_frequently = data[column].value_counts().idxmax()
        # print(most_frequently)
        data[column] = data[column].map({most_frequently: 1, unique_vals[unique_vals != most_frequently][0]: 0})
        print(data[column])

0      1
1      1
2      1
3      0
4      0
      ..
995    1
996    0
997    1
998    0
999    0
Name: Marital Status, Length: 1000, dtype: int64
0      0
1      1
2      1
3      1
4      1
      ..
995    1
996    1
997    1
998    1
999    1
Name: Gender, Length: 1000, dtype: int64
0      1
1      1
2      0
3      1
4      0
      ..
995    1
996    1
997    1
998    0
999    1
Name: Home Owner, Length: 1000, dtype: int64
0      1
1      1
2      1
3      0
4      0
      ..
995    0
996    0
997    0
998    1
999    0
Name: Purchased Bike, Length: 1000, dtype: int64


Удалите остальные категориальные столбцы.

**Вопрос:** сколько категориальных столбцов вы удалили?

In [410]:
columns_to_del = []
for column in data.columns:
    if data[column].dtype == 'object' and data[column].nunique() != 2:
        columns_to_del.append(column)

print(columns_to_del)

data.drop(columns=columns_to_del, inplace=True)
data.head()

['Education', 'Occupation', 'Commute Distance', 'Region']


,ID,Marital Status,Gender,Income,Children,Home Owner,Cars,Age,Purchased Bike
0,12496,1,0,40000,1,1,0,42,1
1,24107,1,1,30000,3,1,1,43,1
2,14177,1,1,80000,5,0,2,60,1
3,24381,0,1,70000,0,1,1,41,0
4,25597,0,1,30000,0,0,0,36,0


Удалите столбец `ID`, так как он по сути нечисловой.

In [411]:
data.drop(columns='ID', inplace=True)
data.head()

,Marital Status,Gender,Income,Children,Home Owner,Cars,Age,Purchased Bike
0,1,0,40000,1,1,0,42,1
1,1,1,30000,3,1,1,43,1
2,1,1,80000,5,0,2,60,1
3,0,1,70000,0,1,1,41,0
4,0,1,30000,0,0,0,36,0


Сформируйте матрицу объект-признак `X` и вектор `y` с целевой переменной.  
Целевая переменная - это последний столбец, `Purchased Bike`.

In [412]:
X = data.drop('Purchased Bike', axis=1)
y = data['Purchased Bike']

Разбейте данные на тренировочную и тестовую часть (`Xtrain`, `Xtest`, `ytrain`, `ytest`), в тест отправьте 30% данных.  
Зафиксируйте `random_state = 42`.

In [413]:
from sklearn.model_selection import train_test_split

Xtrain, Xtest, ytrain, ytest = train_test_split(X, y, train_size=0.7, random_state=42)

**Вопрос:** сколько объектов в матрице `Xtrain`?

In [414]:
print(Xtrain.shape[0])
Xtrain.head()

700


,Marital Status,Gender,Income,Children,Home Owner,Cars,Age
541,1,1,50000,1,1,0,34
440,0,1,90000,0,0,3,34
482,0,1,40000,0,0,0,36
422,0,1,110000,0,0,3,32
778,1,1,90000,5,1,3,41


Почти всё готово для обучения модели!

Осталось отмасштабировать матрицу `X`, так как линейные модели чувствительны к масштабу данных.

*  Обучите на тренировочной матрице (`Xtrain`) `MinMaxScaler` из библиотеки `sklearn.preprocessing`
*  Примените масштабирование и к `Xtrain`, и к `Xtest`
*  Переведите полученные после масштабирования `np.array` обратно в pandas `dataframe`.

Полученные масштабированные матрицы назовите, как и раньше, `Xtrain` и `Xtest`.

In [415]:
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()
scaler.fit(Xtrain)
print(scaler.data_max_)
Xtrain = pd.DataFrame(scaler.transform(Xtrain), columns=Xtrain.columns, index=Xtrain.index)
print(Xtrain)
Xtest = pd.DataFrame(scaler.transform(Xtest), columns=Xtest.columns, index=Xtest.index)
print(Xtest)

[1.0e+00 1.0e+00 1.7e+05 5.0e+00 1.0e+00 4.0e+00 8.9e+01]
     Marital Status  Gender  Income  Children  Home Owner  Cars       Age
541             1.0     1.0  0.2500       0.2         1.0  0.00  0.140625
440             0.0     1.0  0.5000       0.0         0.0  0.75  0.140625
482             0.0     1.0  0.1875       0.0         0.0  0.00  0.171875
422             0.0     1.0  0.6250       0.0         0.0  0.75  0.109375
778             1.0     1.0  0.5000       1.0         1.0  0.75  0.250000
..              ...     ...     ...       ...         ...   ...       ...
106             1.0     1.0  0.3750       0.4         1.0  0.50  0.421875
270             0.0     0.0  0.0000       0.4         1.0  0.00  0.406250
860             0.0     1.0  0.1250       0.0         1.0  0.25  0.109375
435             0.0     0.0  0.0000       0.4         0.0  0.25  0.671875
102             1.0     1.0  0.0000       0.4         1.0  0.00  0.375000

[700 rows x 7 columns]
     Marital Status  Gender  I

Теперь обучите логистическую регрессию на тренировочных данных

In [416]:
from sklearn.linear_model import LogisticRegression

Log_Reg = LogisticRegression()
Log_Reg.fit(Xtrain, ytrain)

LogisticRegression()

Сделайте предсказания на тренировочных и на тестовых данных.

In [417]:
predictions = Log_Reg.predict(Xtest)
print(predictions)

[1 1 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 1 0 1 1 1 1 0 1 0 1 1 1 0 0 1
 0 1 0 1 1 1 1 1 1 0 1 1 0 1 1 1 1 1 1 1 1 1 0 0 0 1 0 1 0 1 1 1 1 1 1 0 1
 0 1 1 1 0 1 1 0 0 0 1 0 1 1 0 1 1 0 0 0 0 1 1 0 1 0 0 0 1 0 1 1 0 0 0 1 0
 0 1 0 0 1 1 1 1 0 1 1 0 0 1 0 1 1 0 1 1 1 1 1 1 1 1 1 0 1 1 1 1 0 0 0 1 1
 1 0 0 1 1 0 0 0 1 1 1 1 1 1 0 1 0 0 1 1 1 0 1 1 0 1 1 1 1 1 1 0 0 0 1 1 1
 1 1 0 1 1 0 0 1 1 1 0 1 0 1 1 1 1 1 0 1 0 1 1 1 1 1 1 0 0 1 1 0 0 1 1 1 0
 0 0 0 1 1 0 0 0 0 1 0 0 0 1 0 1 1 1 0 0 0 0 0 0 1 1 0 1 0 0 1 0 0 0 1 0 1
 1 1 1 1 1 1 1 1 1 1 1 0 1 1 1 1 1 0 0 1 1 1 0 0 0 0 0 1 0 1 0 1 1 0 0 1 1
 1 1 0 1]


Оцените значение accuracy на трейне и на тесте.

In [418]:
from sklearn.metrics import accuracy_score

print(accuracy_score(ytest, predictions))

0.58


Качество модели получилось невысоким, зато модель не переобучена.

Попробуем добавить новых признаков в модель, используя [PolynomialFeatures](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.PolynomialFeatures.html).

Создайте полиномиальные признаки degree = 2.

Как обычно:
*  `fit` делайте на тренировочных данных
*  `transform` и на тренировочных, и на тестовых данных. Затем верните результат к формату pandas `dataframe`.

Полученные матрицы назовите, как и раньше, `Xtrain` и `Xtest`.

In [419]:
from sklearn.preprocessing import PolynomialFeatures

# print(Xtrain.shape)
Pol_Features = PolynomialFeatures(degree=2)
Pol_Features.fit(Xtrain)
Xtrain = pd.DataFrame(Pol_Features.transform(Xtrain), columns=Pol_Features.get_feature_names_out(Xtrain.columns))
# Xtrain.head()
# print(Xtrain.shape)
Xtest = pd.DataFrame(Pol_Features.transform(Xtest), columns=Pol_Features.get_feature_names_out(Xtest.columns))
Xtest.head()

,1,Marital Status,Gender,Income,Children,Home Owner,Cars,Age,Marital Status^2,Marital Status Gender,...,Children^2,Children Home Owner,Children Cars,Children Age,Home Owner^2,Home Owner Cars,Home Owner Age,Cars^2,Cars Age,Age^2
0,1.0,0.0,1.0,0.1875,0.8,1.0,0.5,0.578125,0.0,0.0,...,0.64,0.8,0.4,0.462500,1.0,0.5,0.578125,0.25,0.289062,0.334229
1,1.0,1.0,1.0,0.3750,0.4,0.0,0.5,0.375000,1.0,1.0,...,0.16,0.0,0.2,0.150000,0.0,0.0,0.000000,0.25,0.187500,0.140625
2,1.0,1.0,1.0,0.1875,0.8,0.0,0.0,0.078125,1.0,1.0,...,0.64,0.0,0.0,0.062500,0.0,0.0,0.000000,0.00,0.000000,0.006104
3,1.0,1.0,0.0,0.3125,0.2,1.0,0.0,0.171875,1.0,0.0,...,0.04,0.2,0.0,0.034375,1.0,0.0,0.171875,0.00,0.000000,0.029541
4,1.0,1.0,1.0,0.3750,1.0,1.0,0.5,0.281250,1.0,1.0,...,1.00,1.0,0.5,0.281250,1.0,0.5,0.281250,0.25,0.140625,0.079102


**Вопрос:** на сколько признаков стало больше при добавлении полиномиальных признаков второй степени?

Заново обучите логистическую регрессию, уже на расширенной матрице признаков, и сделайте предсказания на трейне и тесте, а затем оцените качество (*accuracy*).

In [420]:
Log_Reg.fit(Xtrain,ytrain)
predictions = Log_Reg.predict(Xtest)
print(accuracy_score(ytest, predictions))

0.6233333333333333


**Вопрос:** на сколько повысилось качество модели на тестовых данных?  
Ответ округлите до сотых.

Появились новые требования от заказчика!

Заказчик просит, чтобы:
*  доля найденных моделью потенциальных покупателей была максимальной
*  accuracy при этом была не ниже, чем 0.6 (отклонения *accuracy* на тестовых данных на $\pm 0.05$ допустимы).

Сначала посмотрите, какие значения *recall* и *accuracy* имеют предсказния модели на тесте с классами, предсказанными по умолчанию (методом `predict`).

In [421]:
from sklearn.metrics import recall_score
print('recall: ', recall_score(ytest, predictions))
print('accuracy: ', accuracy_score(ytest, predictions))

recall:  0.6959459459459459
accuracy:  0.6233333333333333


In [422]:
best_recall_score = 0
best_threshold = None

for threshold in np.arange(0, 1.001, 0.001):
    probs_test_1 = Log_Reg.predict_proba(Xtest)[:,1]
    predicted_labels = np.array(list(map(lambda x: 1 if x >= threshold else 0, probs_test_1)))

    if recall_score(ytest, predicted_labels) >= best_recall_score and accuracy_score(ytest, predicted_labels) >= 0.56:
        best_recall_score = recall_score(ytest, predicted_labels)
        best_threshold = threshold


print(best_recall_score)
print(best_threshold)

0.8108108108108109
0.41200000000000003


Подберём порог для перевода вероятностей в классы, чтобы оптимизировать требуемые метрики!

Разобъем тренировочные данные на трейн и валидацию, чтобы по валидационной части подбирать порог.

In [423]:
XtrainS, Xval, ytrainS, yval = train_test_split(Xtrain, ytrain, test_size=0.3, random_state=42)

XtrainS.shape, Xval.shape

((490, 36), (210, 36))

* Обучите модель на тренировочных данных.
* Предскажите вероятности положительного класса на валидационных данных

В цикле для каждого значения порога:
*  переведите вероятности в классы
*  вычислите полноту (на валидационных данных)

Выведите на экран:

1) значение порога, дающее максимальный *recall*, при условии, что *accuracy* $\geq$ 0.6.

2) значение *recall* при этом пороге

3) значение *accuracy* для этого порога


Ищите порог на отрезке от 0 до 1 с шагом 0.01.

In [424]:
Log_Reg.fit(XtrainS, ytrainS)
predictions_val = Log_Reg.predict(Xval)
print(predictions_val)

[1 0 0 1 0 1 1 0 0 1 1 0 0 1 0 1 1 1 1 1 1 1 1 1 1 0 1 0 1 0 1 1 1 1 1 1 0
 1 0 0 0 0 0 0 0 1 1 1 0 1 0 0 0 1 0 1 1 1 1 0 0 0 1 1 1 1 0 1 1 0 1 0 0 1
 0 1 1 0 0 0 1 0 1 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 0 0 1 1 1 0 0 1 0 1
 0 0 0 0 1 1 1 1 0 1 1 0 0 1 1 0 1 0 1 1 1 1 1 1 1 0 1 1 1 0 0 0 1 0 1 1 0
 1 1 1 0 1 1 1 1 0 1 1 1 0 1 0 0 1 1 0 0 1 0 1 1 1 1 1 1 1 1 1 1 1 1 1 0 1
 1 1 1 0 0 1 0 0 1 1 0 1 1 1 1 1 1 0 0 0 0 1 1 0 0]


In [425]:
from sklearn.metrics import recall_score

RecMax = -1
BestThr = -1
BestAcc = -1

for thr in np.arange(0, 1, 0.01):
    probs_val_1 = Log_Reg.predict_proba(Xval)[:,1]
    predicted_labels = np.array(list(map(lambda x: 1 if x >= thr else 0, probs_val_1)))

    if recall_score(yval, predicted_labels) >= RecMax and accuracy_score(yval, predicted_labels) >= 0.6:
        RecMax = recall_score(yval, predicted_labels)
        BestThr = thr
        BestAcc = accuracy_score(yval, predicted_labels)

print(BestThr, RecMax, BestAcc)

0.42 0.8518518518518519 0.6142857142857143


Попробуем добавить новых признаков в модель, используя [PolynomialFeatures](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.PolynomialFeatures.html).

Создайте полиномиальные признаки degree = 2.

Как обычно:
*  `fit` делайте на тренировочных данных
*  `transform` и на тренировочных, и на тестовых данных. Затем верните результат к формату pandas `dataframe`.

Полученные матрицы назовите, как и раньше, `Xtrain` и `Xtest`.

Теперь заново обучите модель на исходных тренировочных данных (`Xtrain`, `ytrain`), предскажите вероятности на тесте и переведите их в классы по найденному порогу.

In [426]:
from sklearn.preprocessing import PolynomialFeatures

# print(Xtrain.shape)
Pol_Features = PolynomialFeatures(degree=2)
Pol_Features.fit(Xtrain)
Xtrain = pd.DataFrame(Pol_Features.transform(Xtrain), columns=Pol_Features.get_feature_names_out(Xtrain.columns))
# Xtrain.head()
# print(Xtrain.shape)
Xtest = pd.DataFrame(Pol_Features.transform(Xtest), columns=Pol_Features.get_feature_names_out(Xtest.columns))
Xtest.head()

,1,1,Marital Status,Gender,Income,Children,Home Owner,Cars,Age,Marital Status^2,...,Home Owner Age^2,Home Owner Age Cars^2,Home Owner Age Cars Age,Home Owner Age Age^2,Cars^2^2,Cars^2 Cars Age,Cars^2 Age^2,Cars Age^2,Cars Age Age^2,Age^2^2
0,1.0,1.0,0.0,1.0,0.1875,0.8,1.0,0.5,0.578125,0.0,...,0.334229,0.144531,0.167114,0.193226,0.0625,0.072266,0.083557,0.083557,0.096613,0.111709
1,1.0,1.0,1.0,1.0,0.3750,0.4,0.0,0.5,0.375000,1.0,...,0.000000,0.000000,0.000000,0.000000,0.0625,0.046875,0.035156,0.035156,0.026367,0.019775
2,1.0,1.0,1.0,1.0,0.1875,0.8,0.0,0.0,0.078125,1.0,...,0.000000,0.000000,0.000000,0.000000,0.0000,0.000000,0.000000,0.000000,0.000000,0.000037
3,1.0,1.0,1.0,0.0,0.3125,0.2,1.0,0.0,0.171875,1.0,...,0.029541,0.000000,0.000000,0.005077,0.0000,0.000000,0.000000,0.000000,0.000000,0.000873
4,1.0,1.0,1.0,1.0,0.3750,1.0,1.0,0.5,0.281250,1.0,...,0.079102,0.070312,0.039551,0.022247,0.0625,0.035156,0.019775,0.019775,0.011124,0.006257


**Вопрос:** какое значение *recall* получилось на тестовых данных после подбора порога?  
Ответ округлите до десятых.

При помощи подбора порога удалось сильно увеличить значение *recall*!  
Однако, как видно, на тесте не удалось сохранить условие $accuracy \geq 0.6$ (но в допустимые рамки уложились!)

Это свидетельство небольшого переобучения модели. Однако в сухом остатке имеет значительное увеличение полноты, что является приоритетом для заказчика.